# Exploring Elastic Search

**Mehdi Boustani** - S221594  
**Nicolas Schneiders** - S203005  
**Maxim Piron** - S211493  
**Andreas Stistrup** - S212891  

*Faculty of Applied Sciences, University of Liège*

April 28, 2025


# Introduction

# Installation & configuration

## Docker

### Installing Elasticsearch
If you don't have Docker installed yet, you can download and install it from the [official website](https://www.docker.com/). 

Once Docker is running on your machine, launch Elasticsearch using the following command:

In [ ]:
!docker run -p 127.0.0.1:9200:9200 -d --name elasticsearch \
  -e "discovery.type=single-node" \
  -e "xpack.security.enabled=false" \
  -e "xpack.license.self_generated.type=trial" \
  -v "elasticsearch-data:/usr/share/elasticsearch/data" \
  docker.elastic.co/elasticsearch/elasticsearch:8.15.0


## Dependencies  
Let's install all the necessary Python packages we'll be using throughout this tutorial.


In [ ]:
# requests       → to interact with the Elasticsearch REST API
# elasticsearch  → official Elasticsearch Python client
# pandas         → for handling and analyzing tabular data (e.g., dataset exploration)
# matplotlib     → for optional data visualization (e.g., query stats or aggregations)

!pip install requests elasticsearch==8.15.0 pandas matplotlib

In [ ]:
from pprint import pprint
from elasticsearch import Elasticsearch, helpers

es = Elasticsearch('http://localhost:9200')
info = es.info()

print('Connected to ElasticSearch !')
pprint(info.body)


## Importing data with the bulk api

To efficiently load a large dataset into Elasticsearch, we use the Bulk API. This method allows us to insert multiple documents in a single request, which is much faster and more efficient than indexing documents one by one. In this example, we will import the contents of our `apod.json` file—where each element is a document—into a new index called `apod`.

In [ ]:
import json

with open("apod.json", "r") as f:
    data = json.load(f)

# Prepare the actions for the bulk
actions = [
    {
        "_index": "apod",
        "_id": doc["title"], # We use the title as index since it is a unique field (the unicity is important!)
        "_source": doc
    }
    for doc in data
]

# We import the data in bulk
try:
    helpers.bulk(es, actions)
    print("Bulk import terminé !")
except  Exception as e:
    print(e)

## Basic queries in ElasticSearch

Elasticsearch exposes a RESTful API, which means you interact with it using standard HTTP methods. Here are the most common operations:

- **GET**: Read a document or perform a search
- **POST**: Add a new document
- **PUT**: Create or replace a document or an index
- **DELETE**: Remove a document or an index

### GET method

The GET method is used to retrieve data from our json file by providing an id. If the document with the specified id doesn't exist, it throws an exception.

In [15]:
try:
    doc = es.get(index="apod", id="A Hazy Harvest Mofffon")
    pprint(doc['_source'])

except:
    print("A document with this id doesn't exist!")

A document with this id doesn't exist!


### POST method

The POST method is used to create a new document. When using the index() method without specifying an id, elasticsearch automatically generates one (not the title as the other documents).

In [26]:
from datetime import datetime

new_id = "A New APOD"

new_doc = {
    "date": datetime.now().strftime("%Y-%m-%d"),
    "title": new_id,
    "explanation": "This is a new document added via POST.",
    "image_url": "https://apod.nasa.gov/apod/image/2410/new_apod.jpg",
    "authors": "Mehdi Boustani"
}

res = es.index(index="apod", document=new_doc)

print("Document added successfully")

Document added successfully


### PUT method

The PUT method is used to create or replace a document at a specified id. If a document with that id already exists, it will be overwritten.

In [37]:
replaced_id = "Replaced APOD"

doc = {
    "date": "2024-10-02",
    "title": replaced_id,
    "explanation": "This document replaces any previous one with the same ID.",
    "image_url": "https://apod.nasa.gov/apod/image/2410/new_apod.jpg",
    "authors": "Mehdi Boustani"
}

# Let's replace our previously created document
es.index(index="apod", id=new_id, document=doc)

# Since we use personnalized id, let's replace it manually
es.index(index="apod", id=replaced_id, document=doc)

print(f"Document with id '{new_id}' replaced by a new document with id '{replaced_id}'")

Document with id 'A New APOD' replaced by a new document with id 'Replaced APOD'


### DELETE method

This method is used to delete a document by its id. The id must be known and specified in the request.

In [38]:
try:
    es.delete(index="apod", id=replaced_id)
    print(f"Document with ID {replaced_id} deleted.")

except:
    print("The specified document to delete doesn't exist")

# Delete the entire index (Attention: this is irreversible!)
# es.indices.delete(index="apod")
# print("Index 'apod' deleted.")

Document with ID Replaced APOD deleted.


# Conclusion

# References

1. <a id="freecodecamp"></a> [Elasticsearch Course for Beginners - FreeCodeCamp](https://www.youtube.com/watch?v=a4HBKEda_F8&ab_channel=freeCodeCamp.org)

2. <a id="elasticdoc"></a> [Elastic Official Documentation](https://www.elastic.co/docs/get-started)

3. <a id="elasticlab"></a> [Elastic Search Lab - Tutorials](https://www.elastic.co/search-labs/tutorials)

# Process and Work Distribution
